[View in Colaboratory](https://colab.research.google.com/github/jolivaresc/Reinforcement_Learning_for_Stock_Prediction/blob/master/reinforcement_trading.ipynb)

In [28]:
! pip install gym

In [29]:
!git clone https://github.com/jolivaresc/Reinforcement_Learning_for_Stock_Prediction.git

fatal: destination path 'Reinforcement_Learning_for_Stock_Prediction' already exists and is not an empty directory.


In [30]:
!cd Reinforcement_Learning_for_Stock_Prediction/ && ls

agent  data  evaluate.py  functions.py	images	model  README.md  train.py


In [0]:
!cd Reinforcement_Learning_for_Stock_Prediction/ && mkdir models

In [35]:
!cd Reinforcement_Learning_for_Stock_Prediction/ && ls

agent  evaluate.py   images  models	  README.md
data   functions.py  model   __pycache__  train.py


In [0]:
!cd Reinforcement_Learning_for_Stock_Prediction/model && wget -qO- --keep-session-cookies --save-cookies cookies.txt https://finance.yahoo.com/quote/ARDM/history?p=ARDM && wget wget -qO- --load-cookies cookies.txt https://query1.finance.yahoo.com/v7/finance/download/ARDM?period1=1504030392&period2=1506708792&interval=1d&events=history&crumb=TKe9axyOsuR

In [0]:
!cd Reinforcement_Learning_for_Stock_Prediction/ && python train.py ^GSPC 10 1000

Using TensorFlow backend.
Episode 0/1000
Buy: $1347.56
Sell: $1326.82 | Profit: -$20.74
Buy: $1318.55
Sell: $1326.65 | Profit: $8.10
Buy: $1329.47
Sell: $1342.54 | Profit: $13.07
Buy: $1357.51
Buy: $1364.17
Sell: $1373.73 | Profit: $16.22
Sell: $1373.47 | Profit: $9.30
Buy: $1354.31
Sell: $1352.26 | Profit: -$2.05
Buy: $1340.89
Sell: $1330.31 | Profit: -$10.58
2018-10-05 02:29:56.931774: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-10-05 02:29:56.932234: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1411] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-10-05 02:29:56.932274: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1490] Adding visible gpu devices: 0
2018-10-05 02:29:57.318847: I tensorflow/core/common_ru

In [0]:
# -*- coding: utf-8 -*-
import random
import gym
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, RMSprop
import tensorflow as tf

print(f"TensorFlow v{tf.VERSION}")
if tf.test.gpu_device_name():
  print("GPU disponible")

EPISODES = 200
OPT = Adam

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(30, input_dim=self.state_size, activation='tanh'))
        model.add(Dense(30, activation='tanh'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=OPT(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


In [0]:
if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-dqn.h5")
    done = False
    batch_size = 32

    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            #env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print(f"episode: {e}/{EPISODES}, score: {time}, e: {agent.epsilon}")
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
        # if e % 10 == 0:
        #     agent.save("./save/cartpole-dqn.h5")